In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pygsp import graphs, filters, plotting
import pygsp as pg
from numpy.random import default_rng
from numpy.random import randint
from random import sample
import random
import cvxpy as cp

plotting.BACKEND = 'matplotlib'
plt.rcParams['figure.figsize'] = (10, 5)

In [2]:
def createGraph(Nodes,i,graphType):
    while True:
        s = np.array(sample(range(1, 10000), 1))
        G = pg.graphs.Sensor(N=Nodes, seed = s)
        typeName = 'Sensor'
            
        if (G.is_connected() and not(G.is_directed())):
            break
    if i==0:        
        print(typeName)
           
    G.compute_fourier_basis()
    W = np.random.uniform(size=(2,1))
    x =  G.U[:,1:3]@W
    x = x/np.max(np.abs(x))
    eigenvalue = G.e
    print(eigenvalue[1:4])

    return G,x

In [3]:
def computeMainSignal(signals,k,N):
    x = np.zeros((N,1))
    for i in range(k):
        x = x + signals[:,i].reshape(N,1)
    return x

In [4]:
def computeSmoothness(signals,graphs,k):
    smoothness = []
    sumOverSmoothness = 0
    for i in range(k):
        smoothness.append(signals[:,i].T@graphs[i].L@signals[:,i])
        sumOverSmoothness = sumOverSmoothness + smoothness[i]
    return sumOverSmoothness,smoothness

In [5]:
def decomposeGraphSignal(mainSignal,signals,graphs,k,N):
    A = np.tile(np.identity(N), (1, k))
    
    H = np.zeros((N*k,N*k))
    for i in range(k):
        g = graphs[i].L
        H[i*N:(i+1)*N,i*N:(i+1)*N] = g.toarray()
        
        
    X = cp.Variable((N*k,1))
    objective = cp.Minimize(cp.quad_form(X, H)) 
    
    constraints = [ A @ X == mainSignal]
    prob = cp.Problem(objective, constraints)
    prob.solve(verbose=True)
    
    temp = X.value
    X_hat = temp.reshape(N,k, order='F')
    
    return X_hat

In [6]:
def makeMeanZero(signals,k):
    for i in range(k):
            signals[:,i] = signals[:,i] - np.mean(signals[:,i])
            
    return signals

In [7]:
def calculateSNR(est_signals,signals,k):
    SNR = np.zeros((1,k))
    for i in range(k):
        SNR[0,i]=(10*np.log10(np.linalg.norm(signals[:,i])**2/np.linalg.norm(signals[:,i]-est_signals[:,i])**2))
    return SNR

In [8]:
N = 250  # number of nodes
k = 5    # number of graphs
np.random.seed(0)
random.seed(0)
iteration = 200
av_snr = np.zeros((1,k))

for itera in range(1,iteration+1):
    print('############################################# iteration = ',itera,' ########################################')
    # create graph and graph signals
    graphType = randint(1, 3, 1)
    graph_list = []
    signals = np.zeros((N,k))
    for i in range(k):
        G , x = createGraph(N,i,graphType)
        graph_list.append(G)
        signals[:,[i]] = x

    # calculate the main signal : x = x1+x2+...+x4
    mainSignal = computeMainSignal(signals,k,N)
    mainSignal = mainSignal - np.mean(mainSignal)

    # decompose graph signal
    estimated_signals = decomposeGraphSignal(mainSignal,signals,graph_list,k,N)

    # calculate graph signals' smoothness
    sumOverSmoothness,smoothness = computeSmoothness(signals,graph_list,k)
    print('smoothness:')
    print(smoothness)
    print('sum over smoothness:',sumOverSmoothness)

    # calculate estimated graph signals' smoothness
    sumOverSmoothness_est,smoothness_est = computeSmoothness(estimated_signals,graph_list,k)
    print('smoothness_est:')
    print(smoothness_est)
    print('sum over smoothness_est:',sumOverSmoothness_est)

    # make mean of the estimated signals zero to remove DC values
    estimated_signals = makeMeanZero(estimated_signals,k)

    # calculate SNRs
    SNRs = calculateSNR(estimated_signals,signals,k)
    print('SNR:',SNRs)
    av_snr = av_snr + SNRs/iteration

    # compute objective values
    objectiveValue = sumOverSmoothness
    objectiveValue_est = sumOverSmoothness_est
    print('objectiveValue:',objectiveValue)
    print('objectiveValue_est:',objectiveValue_est)

    # compute error
    error = np.linalg.norm(mainSignal-estimated_signals@np.ones((k,1)))
    print('error:',error)
    
    
print('av_snr: ',av_snr)
print('total_av_snr: ',np.mean(av_snr))

############################################# iteration =  1  ########################################
Sensor
[0.09819918 0.13965542 0.16814379]
[0.08541946 0.10730097 0.19473386]
[0.07636719 0.12163677 0.18823118]
[0.08208824 0.10205125 0.14759177]
[0.08055813 0.1055017  0.15590231]
-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 1250, constraints m = 250
          nnz(P) + nnz(A) = 9531
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: o

2021-12-03 14:50:20,252:[WARNING](pygsp.graphs.sensor.__init__): Graph is not connected.


[0.06557593 0.09028861 0.15870866]
[0.08639341 0.12378437 0.19321071]
[0.1037146  0.10600691 0.19469326]
[0.11782776 0.12662681 0.19316097]
-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 1250, constraints m = 250
          nnz(P) + nnz(A) = 9406
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rho        time
   1   0.0000e+00   3.28e+00 

smoothness:
[3.2744446453234826, 3.7095383494076053, 4.680737937420424, 4.880300272850462, 9.483946186746087]
sum over smoothness: 26.02896739174806
smoothness_est:
[3.210031712618238, 3.7163880303752626, 4.40829238574913, 4.33067736575394, 8.941727325568817]
sum over smoothness_est: 24.607116820065386
SNR: [[23.16960777 24.5870511  23.05565318 20.7673194  26.41223936]]
objectiveValue: 26.02896739174806
objectiveValue_est: 24.607116820065386
error: 3.83540809174956e-15
############################################# iteration =  8  ########################################
Sensor
[0.09752216 0.14432902 0.24682651]
[0.09135297 0.12429003 0.18001639]
[0.07929894 0.11707833 0.19021406]
[0.06660332 0.11367299 0.15993293]
[0.06619359 0.11432317 0.18293618]
-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
--------

2021-12-03 14:50:35,281:[WARNING](pygsp.graphs.sensor.__init__): Graph is not connected.


...

2021-12-03 14:54:42,739:[WARNING](pygsp.graphs.sensor.__init__): Graph is not connected.


[0.084995   0.11993401 0.19403871]
[0.07765116 0.10908675 0.17884088]
[0.08288292 0.08975832 0.20593643]
-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 1250, constraints m = 250
          nnz(P) + nnz(A) = 9291
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rho        time
   1   0.0000e+00   2.94e+00   3.67e+03   1.00e-01   1.41e-01s
 

Sensor
[0.07557608 0.11580752 0.13124151]
[0.0874899  0.10102821 0.18044912]
[0.09587157 0.1167639  0.22185945]
[0.07811925 0.09202005 0.17551776]
[0.08474475 0.11724491 0.20495273]
-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 1250, constraints m = 250
          nnz(P) + nnz(A) = 9548
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rho

[0.07040376 0.09618938 0.15234906]
[0.11209245 0.12357051 0.1811622 ]
[0.05279168 0.10793208 0.19347778]
[0.10275116 0.12249664 0.20618437]
-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 1250, constraints m = 250
          nnz(P) + nnz(A) = 9471
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rho        time
   1   0.0000e+00   3.35e+00 

[0.10285858 0.12014942 0.21104985]
[0.07749661 0.11737327 0.16107392]
[0.07239517 0.1156114  0.19581658]
[0.08536874 0.116432   0.20873613]
[0.07525091 0.11709378 0.21778554]
-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 1250, constraints m = 250
          nnz(P) + nnz(A) = 9470
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rho       